In [ ]:
import numpy as np
from scipy.special import jv
import matplotlib.pyplot as plt

# Physical parameters of the reflector antenna
diameter = 1.0   # m
focal_length = 0.5 * diameter   # m
feed_position = 0.25 * diameter   # m
frequency = 10e9   # Hz

# Calculation of reflector surface shape
x = np.linspace(-diameter/2, diameter/2, 101)
y = np.linspace(-diameter/2, diameter/2, 101)
xx, yy = np.meshgrid(x, y)
zz = -(xx**2 + yy**2) / (4*focal_length)

# Calculation of field distribution on reflector surface
k = 2*np.pi*frequency / 299792458.0   # wavenumber
a = diameter / 2   # aperture radius
b = feed_position   # distance from center to feed
theta = np.linspace(0.000001, np.pi, 181)
phi = np.linspace(0.000001, 2*np.pi, 361)
theta, phi = np.meshgrid(theta, phi)
d = focal_length - np.sqrt(focal_length**2 - a**2)
x = d / np.tan(theta)
rho = np.sqrt(x**2 + d**2 + a**2 + b**2 - 2*a*x*np.sin(phi) - 2*b*d*np.cos(phi))
pattern = a**2 * (jv(1, k * rho * np.sin(theta)) / (k * rho * np.sin(theta)))**2
pattern = np.nan_to_num(pattern)

# Calculation of radiation pattern
r = np.sqrt(xx**2 + yy**2 + zz**2)
theta = np.arccos(zz / r)
phi = np.arctan2(yy, xx)
E = np.zeros_like(r, dtype=complex)
for i in range(len(theta)):
    for j in range(len(phi)):
        if r[i,j] > 1e-10:
            kx = k * r[i,j] * np.sin(theta[i,j]) * np.cos(phi[j])
            ky = k * r[i,j] * np.sin(theta[i,j]) * np.sin(phi[j])
            kz = k * r[i,j] * np.cos(theta[i,j])
            E[i,j] = np.exp(-1j * (kx*x + ky*y + kz*focal_length)) * pattern[i,j] * np.sin(theta[i,j])
E = np.nan_to_num(E)
E_theta = np.abs(np.sum(E * np.sin(theta) * np.cos(phi)))**2
E_phi = np.abs(np.sum(E * np.sin(theta) * np.sin(phi)))**2
gain = 10 * np.log10(4 * np.pi * (E_theta + E_phi) / (np.pi * diameter)**2)

# Plot the radiation pattern
plt.plot(theta, gain)
plt.title('Radiation Pattern of a Reflector Antenna')
plt.xlabel('Angle (radians)')
plt.ylabel('Gain (dBi)')
plt.ylim(-60, 10)
plt.grid(True)
plt.show()

## NEXT


import numpy as np
import matplotlib.pyplot as plt
from scipy.special import jv
from math import pi

# User prompt for choosing azimuth or elevation cut
cut_type = input("Choose cut type (azimuth/elevation): ")

# Specify the dimensions and frequency of the antenna
d = 2.0
f = 5.0e9

# Specify the dimensions of the reflector
a = 10.0
b = 8.0

# Define the angle range for the plot
if cut_type.lower() == "azimuth":
    theta = np.linspace(-90, 90, 361)
    phi = 0.0
elif cut_type.lower() == "elevation":
    theta = np.linspace(0, 180, 361)
    phi = 90.0
else:
    print("Invalid cut type. Please choose either 'azimuth' or 'elevation'.")
    exit()

# Calculate the radiation pattern
pattern = []
for th in theta:
    if cut_type.lower() == "azimuth":
        x = d * np.tan(th * pi / 180.0)
        rho = np.sqrt(x**2 + d**2)
    elif cut_type.lower() == "elevation":
        th_rad = th * pi / 180.0
        if np.abs(np.tan(th_rad)) < 1e-10:
            x = d / (1e-10)
        else:
            x = d / np.tan(th_rad)

    rho = np.sqrt(np.where(x**2 + d**2 + a**2 + b**2 - 2*a*x*np.sin(phi*pi/180.0) - 2*b*d*np.cos(phi*pi/180.0) >= 0,
                        x**2 + d**2 + a**2 + b**2 - 2*a*x*np.sin(phi*pi/180.0) - 2*b*d*np.cos(phi*pi/180.0),
                        0))
    k = 2 * pi * f / 3e8
    a = np.sqrt(a**2 + b**2)
    pattern.append(10 * np.log10(a**2 * (jv(1, k * rho * np.sin(phi*pi/180.0)) / (k * rho * np.sin(phi*pi/180.0) + 1e-9))**2))

# Plot the radiation pattern
plt.plot(theta, pattern)
plt.xlabel(cut_type.title() + " Cut Angle (degrees)")
plt.ylabel("Gain (dBi)")
plt.title("Radiation Pattern of a Reflector Antenna")
plt.show()



In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

# Define the geometry of the reflector antenna
radius = 1.0 # meters
aperture_diameter = 2.0 * radius # meters
focal_length = 2.0 * radius # meters
feed_position = np.array([0.0, 0.0, focal_length]) # feed position in z axis (meters)

# Define the radiation pattern of the feed
theta = np.linspace(0, np.pi, 91)
phi = np.linspace(0, 2*np.pi, 181)

# Avoid division by zero in future
if theta[0] == 0:
    theta[0] = 1e-5
if phi[0] == 0:
    phi[0] = 1e-5
        
theta, phi = np.meshgrid(theta, phi, indexing='ij')

# Example feed radiation pattern: isotropic
feed_pattern = np.ones_like(theta)

# Calculate the phase shift and amplitude taper of the feed
phase_shift = np.zeros_like(theta)
amplitude_taper = np.ones_like(theta)

# Use the geometry and radiation pattern of the feed to calculate the aperture field
wavelength = 1.0 # meters
k = 2*np.pi / wavelength
aperture_radius = aperture_diameter / 2
aperture_field = feed_pattern * amplitude_taper * np.exp(-1j*k*focal_length) * np.exp(1j*k*aperture_radius*np.sin(theta)*np.cos(phi))

# Calculate the radiation pattern of the reflector antenna
antenna_pattern = np.abs(np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(aperture_field))))**2
antenna_pattern_dBi = 10*np.log10(antenna_pattern/np.max(antenna_pattern))

# Normalize the radiation pattern to the maximum value
antenna_pattern_dBi = antenna_pattern_dBi - np.max(antenna_pattern_dBi)

# Plot the radiation pattern
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='polar')
r, theta = np.meshgrid(np.linspace(0, 1, 91), np.linspace(0, 2*np.pi, 181), indexing='ij')
ax.pcolormesh(theta, r, antenna_pattern_dBi)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import jv

# Antenna dimensions
D = 1.0  # Diameter
f = 10.0  # Focal length
L = 0.5  # Distance from feed to vertex

# Frequencies to calculate pattern
freq = np.linspace(1, 10, 100)

# Wavelength and wave number
c = 299792458.0
lambda_ = c / freq
k = 2 * np.pi / lambda_

# Antenna aperture
r = np.linspace(-D / 2, D / 2, 101)
theta = np.linspace(0, np.pi, 181)* np.pi / 180+1e-5
phi = np.arange(0, 361, 1) * np.pi / 180 +1e-5
r, theta = np.meshgrid(r, theta)

# Parabolic reflector shape
zz = L**2 / (4 * f)
rr = np.sqrt(r**2 + zz**2)
x = r * np.sin(theta)
y = r * np.cos(theta)
z = np.sqrt(rr**2 - x**2 - y**2)

# Calculate the pattern for each frequency and azimuth cut
pattern = np.zeros((len(freq), len(theta), len(phi)), dtype=np.complex128)
for i in range(len(freq)):
    for j in range(len(phi)):
        a_eff = np.pi * D**2 / 4  # Effective aperture
        k_r = k[i] * r
        k_z = k[i] * z
        a = np.sin(theta) * np.cos(phi[j])
        b = np.sin(theta) * np.sin(phi[j])
        d = np.cos(theta)
        x = np.where(theta == 0, 0, d / np.tan(theta))
        rho = np.sqrt(x**2 + d**2 + a**2 + b**2 - 2*a*x*np.sin(phi[j]) - 2*b*d*np.cos(phi[j]))
        eps = 1e-10
        a_pattern = jv(1, k_r * np.sin(phi[j])) / (k_r * np.sin(phi[j]) + eps)
        b_pattern = np.exp(-1j * k_z)
        pattern[i,:,j] = a_eff * (a_pattern * b_pattern)**2
        if i > 0:
            pattern[i,:,j] += pattern[i-1,:,j]


# Plot the radiation pattern
theta_vals = np.linspace(0, 180, len(pattern[0]))
pattern = pattern.reshape(-1)
plt.figure()
plt.plot(theta_vals, pattern)
plt.xlabel('Theta (degrees)')
plt.ylabel('Gain (dBi)')
plt.show()
